In [4]:
import scipy.sparse as sp
import pandas as pd
import numpy as np


class Dataset(object):
    def __init__(self, path):
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)
        
        self.num_users, self.num_items = self.trainMatrix.shape

    def load_rating_file_as_list(self, filename):
        df = pd.read_csv(filename, sep="\t")
        ratingList = list(zip(df.userid.tolist(), df.itemid.tolist()))
        return ratingList
    
    def load_negative_file(self, filename):
        df = pd.read_csv(filename, sep="\t")
        negativeList = df.iloc[:, 1:].values.tolist()
        return negativeList
    
    def load_rating_file_as_matrix(self, filename):
        df = pd.read_csv(filename, sep="\t")
        num_users = df.userid.max()
        num_items = df.itemid.max()
        mat = sp.dok_matrix((num_users + 1, num_items + 1), dtype=np.float32)
        interactions = df[['userid', 'itemid']].values.tolist()
        # [(0, 2969), (0, 1178), (0, 1574), (0, 957)]
        for user, item in interactions:
            mat[user, item] = 1.
        # [((0, 2969), 1.0), ((0, 1178), 1.0), ((0, 1574), 1.0), ((0, 957), 1.0)]
        return mat

Data_Javier/ml-1m.test.negative
    positive  item_n0  item_n1  item_n2  item_n3  item_n4  item_n5  item_n6  \
0    (0, 47)     3184     3478     1694     3377      340     2111     2833   
1  (1, 1737)      683       15      369     1630      777      899     2465   
2  (2, 1900)      478     3184     2477     1061     2196      416      976   
3  (3, 1774)     1085     2309     1203     3366     3661     3241     2349   
4   (4, 279)      476     1002      596     2478     2382     2791      704   

   item_n7  item_n8  ...  item_n89  item_n90  item_n91  item_n92  item_n93  \
0      382     2152  ...      2486       428       416       799      2623   
1     2906     2175  ...       235      3515      3255      1878      2017   
2     1623     3263  ...      1492      1994      2294      3431      2735   
3     2363     2903  ...      1768       752      2217      1222        86   
4     1097      624  ...      1527        78       958      2780      3703   

   item_n94  item_n95  i

In [5]:
import math
import heapq
import numpy as np


def evaluate_model(model, testRatings, testNegatives, topK):

    global _model
    global _testRatings
    global _testNegatives
    global _topK

    _model = model
    _testRatings = testRatings
    _testNegatives = testNegatives
    _topK = topK

    hits, ndcgs = [],[]
    for idx in range(len(_testRatings)):
        (hr,ndcg) = eval_one_rating(idx)
        hits.append(hr)
        ndcgs.append(ndcg)
    return (hits, ndcgs)


def eval_one_rating(idx):
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    predictions = _model.predict([users, np.array(items)],
                                 batch_size=100, verbose=0)

    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()

    ranklist = heapq.nlargest(_topK, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)


def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0


def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

def get_train_instances(train, n_items, n_neg, testNegatives):
    user, item, labels = [],[],[]
    n_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user.append(u)
        item.append(i)
        labels.append(1)
        # negative instances: we also need to make sure they are not in the
        # test dataset
        for t in range(n_neg):
            j = np.random.randint(n_items)
            while ((u, j) in train.keys()) or (j in testNegatives[u]):
                j = np.random.randint(n_items)
            user.append(u)
            item.append(j)
            labels.append(0)
    return np.array(user), np.array(item), np.array(labels)

In [15]:
import numpy as np
import pandas as pd
import os
import heapq
import tensorflow.keras
import multiprocessing as mp
from tensorflow.keras import initializers
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Dense, Embedding, Input, Flatten, multiply
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l2

from time import time

def GMF(n_users, n_items, n_emb, reg):
    user = Input(shape=(1,), dtype='int32', name='user_input')
    item = Input(shape=(1,), dtype='int32', name='item_input')

    # user and item embeddings
    # [?, 1, 8]
    MF_Embedding_User = Embedding(
        input_dim=n_users,
        output_dim=n_emb,
        name='user_embedding',
        embeddings_initializer='normal',
        embeddings_regularizer=l2(reg),
        input_length=1)
    # [?, 1, 8]
    MF_Embedding_Item = Embedding(
        input_dim=n_items,
        output_dim=n_emb,
        name='item_embedding',
        embeddings_initializer='normal',
        embeddings_regularizer=l2(reg),
        input_length=1)

    # Flatten and multiply
    # [?, 8]
    user_latent = Flatten()(MF_Embedding_User(user))
    # [?, 8]
    item_latent = Flatten()(MF_Embedding_Item(item))
    # [?, 8]
    predict_vector = multiply([user_latent, item_latent])

    # output layer
    prediction = Dense(1, activation='sigmoid',
                       kernel_regularizer=l2(reg),
                       kernel_initializer='lecun_uniform',
                       name='prediction')(predict_vector)

    # Model
    model = Model(inputs=[user, item], outputs=prediction)

    return model


if __name__ == '__main__':
    datadir = "Data_Javier/"
    dataname = "ml-1m"
    modeldir = "models"
    n_emb = 8
    reg = 0.0
    batch_size = 256
    epochs = 20
    learner = "adam"
    lr = 0.01
    validate_every = 1
    save_model = True
    topK = 10
    n_neg = 4

    modelfname = "keras_GMF" + \
                 "_".join(["_bs", str(batch_size)]) + \
                 "_".join(["_reg", str(reg).replace(".", "")]) + \
                 "_".join(["_lr", str(lr).replace(".", "")]) + \
                 "_".join(["_n_emb", str(n_emb)]) + ".h5"
    modelpath = os.path.join(modeldir, modelfname)
    resultsdfpath = os.path.join(modeldir, 'results_df.p')

    dataset = Dataset(os.path.join(datadir, dataname))
    train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
    n_users, n_items = train.shape

    model = GMF(n_users, n_items, n_emb, reg)
    if learner.lower() == "adagrad":
        model.compile(optimizer=Adagrad(lr=lr), loss='binary_crossentropy')
    elif learner.lower() == "rmsprop":
        model.compile(optimizer=RMSprop(lr=lr), loss='binary_crossentropy')
    elif learner.lower() == "adam":
        model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy')
    else:
        model.compile(optimizer=SGD(lr=lr), loss='binary_crossentropy')

    best_hr, best_ndcg, best_iter = 0, 0, 0
    for epoch in range(1, epochs + 1):
        t1 = time()
        user, item, labels = get_train_instances(train, n_items, n_neg, testNegatives)
        hist = model.fit([user, item], labels, batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
        t2 = time()
        if epoch % validate_every == 0:
            (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK)
            hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
            print("Iteration {}: {:.2f}s, HR = {:.4f}, NDCG = {:.4f}, loss = {:.4f}, validated in {:.2f}s"
                  .format(epoch, t2 - t1, hr, ndcg, loss, time() - t2))
            if hr > best_hr:
                best_hr, best_ndcg, best_iter, train_time = hr, ndcg, epoch, t2 - t1
                if save_model:
                    model.save_weights(modelpath, overwrite=True)

    print("End. Best Iteration {}:  HR = {:.4f}, NDCG = {:.4f}. "
          .format(best_iter, best_hr, best_ndcg))
    if save_model:
        print("The best GMF model is saved to {}".format(modelpath))

Iteration 1: 81.23s, HR = 0.5623, NDCG = 0.3145, loss = 0.3356, validated in 11.75s
Iteration 2: 80.64s, HR = 0.6005, NDCG = 0.3359, loss = 0.2996, validated in 11.56s
Iteration 3: 80.51s, HR = 0.6063, NDCG = 0.3403, loss = 0.2922, validated in 12.16s
Iteration 4: 81.14s, HR = 0.6056, NDCG = 0.3404, loss = 0.2899, validated in 11.77s
Iteration 5: 80.82s, HR = 0.6238, NDCG = 0.3542, loss = 0.2852, validated in 11.59s
Iteration 6: 81.33s, HR = 0.6252, NDCG = 0.3553, loss = 0.2827, validated in 11.77s
Iteration 7: 80.70s, HR = 0.6233, NDCG = 0.3528, loss = 0.2818, validated in 11.49s
Iteration 8: 80.91s, HR = 0.6432, NDCG = 0.3646, loss = 0.2817, validated in 11.76s
Iteration 9: 81.02s, HR = 0.6301, NDCG = 0.3608, loss = 0.2813, validated in 11.92s
Iteration 10: 80.85s, HR = 0.6320, NDCG = 0.3624, loss = 0.2813, validated in 11.53s
Iteration 11: 80.80s, HR = 0.6407, NDCG = 0.3657, loss = 0.2817, validated in 11.42s
Iteration 12: 79.89s, HR = 0.6040, NDCG = 0.3455, loss = 0.2820, validated